# Combining dfs and making dfs for basins

### Import things

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
sam = pd.read_csv('sams_clean_data.csv')
kat = pd.read_csv('basin_2000_2010.csv')
grant = pd.read_csv('90s-data.csv')

In [3]:
try:
    os.mkdir('./basin_csvs')
except:
    pass

### Making all the columns the same

In [4]:
sam.columns == kat.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [5]:
grant.columns

Index(['YYYYMMDD', 'lat', 'long', 'basin_name', 'station_id', 'acton_id',
       'station_name', 'elevation', 'wteq_amt', 'wteq_med', 'wteq_amt_pct_med',
       'prec_wytd_amt', 'prec_wytd_avg', 'prec_wytd_pctavg'],
      dtype='object')

In [6]:
dummies = pd.get_dummies(grant['basin_name'])

In [7]:
grant = pd.concat([grant, dummies], axis=1)

In [8]:
grant.drop('basin_name', axis=1, inplace=True)

In [9]:
grant.columns = grant.columns.str.lower()
kat.columns = kat.columns.str.lower()
sam.columns = sam.columns.str.lower()

In [10]:
grant.columns == sam.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

### Making seperate dfs for each basin

In [11]:
#first combine all our dfs
df = pd.concat([sam, grant, kat])

In [12]:
df.to_csv('allyears.csv')

In [13]:
df.columns = df.columns.str.replace(",",'').str.replace(' ', '_')

In [14]:
basin_names = df.drop(['yyyymmdd', 'lat', 'long', 'station_id', 'acton_id', 'station_name',
                       'elevation', 'wteq_amt', 'wteq_med', 'wteq_amt_pct_med',
                       'prec_wytd_amt', 'prec_wytd_avg', 'prec_wytd_pctavg'], axis=1).columns

In [15]:
basin_names

Index(['big_and_little_lost_basins', 'big_and_little_wood_basins',
       'bitterroot', 'chelan_entiat_wenatchee', 'clearwater_and_salmon',
       'columbia_above_methow', 'deschutes_crooked_john_day',
       'flathead_river_basin', 'grand_ronde_powder_burnt_imnaha',
       'henrys_fork_teton_willow_blackfoot_portneuf', 'idaho_panhandle_region',
       'kootenai_river_in_montana', 'lewis_cowlitz',
       'lower_clark_fork_river_basin', 'lower_columbia_hood_river',
       'owyhee_malheur', 'raft_goose_salmon_falls_bruneau',
       'snake_above_palisades', 'umatilla_walla_walla_willow',
       'upper_clark_fork_river_basin', 'weiser_payette_boise',
       'white_green_cedar_skykomish_snoqualmi_baker_skagit', 'willamette',
       'yakima_ahtanum'],
      dtype='object')

#### Function for seperating out basins

In [16]:
def make_list(basin):
    l = []
    sub_df = df[df[basin] == 1]
    l.append(sub_df)
    return l

In [17]:
basin_dict = {}
for name in basin_names:
    basin_dict[name] = make_list(name)

In [18]:
for key in basin_dict:
    basin_df = basin_dict[key][0]
    basin_df.reset_index(drop=True, inplace=True)
    basin_df.to_csv(f'./basin_csvs/{key}.csv')